In [1]:
import pandas as pd
from sklearn.metrics import classification_report

In [2]:
# From 2_collect_ground_truth_partisanship_annotations_to_dataframe.ipynb
ground_truth = pd.read_pickle("partisanship_annotated_article_ids.pickle")
print(ground_truth.head(30))
print()
print(ground_truth.hyperpartisan.value_counts())

         id hyperpartisan          bias
0   0000017         false   left-center
1   0000023          true         right
2   0000029          true          left
3   0000032          true         right
4   0000037         false  right-center
5   0000070          true          left
6   0000077         false   left-center
7   0000079         false         least
8   0000092          true         right
9   0000101          true         right
10  0000144         false   left-center
11  0000175          true         right
12  0000177          true         right
13  0000195         false         least
14  0000202          true          left
15  0000217         false   left-center
16  0000226         false   left-center
17  0000232         false   left-center
18  0000233          true         right
19  0000242          true          left
20  0000250          true          left
21  0000254         false         least
22  0000255          true          left
23  0000280         false         least


In [3]:
discoursebias = pd.read_pickle("biasindex_scored_paragraphed_article_validation_bypublisher_20181122.pickle")
print(discoursebias.head(3))
print()
print(discoursebias.bias.value_counts())

        id         bias
0  0000017  Diversified
1  0000023  Diversified
2  0000029  Diversified

Diversified    126301
Focused         18335
Dispersed        5035
Biased            329
Name: bias, dtype: int64


In [4]:
def calc_hyperpartisanish(bias):
    hyperpartisanish = None
    if bias == "Dispersed" or bias == "Diversified":
        hyperpartisanish = "false" # hyperpartisan true or false
    else:
        hyperpartisanish = "true" # hyperpartisan true or false
    return hyperpartisanish

In [5]:
discoursebias["hyperpartisanish"] = discoursebias.bias.apply(calc_hyperpartisanish)
print(discoursebias.hyperpartisanish.value_counts())

false    131336
true      18664
Name: hyperpartisanish, dtype: int64


In [6]:
discoursebias["hyperpartisan_gt"] = ground_truth["hyperpartisan"] # column hyperpartisan_gt is ground truth

In [16]:
result = classification_report(discoursebias["hyperpartisanish"],discoursebias["hyperpartisan_gt"])

In [17]:
print(result)

              precision    recall  f1-score   support

       false       0.83      0.47      0.60    131336
        true       0.08      0.32      0.13     18664

   micro avg       0.45      0.45      0.45    150000
   macro avg       0.45      0.40      0.37    150000
weighted avg       0.74      0.45      0.54    150000



Since the above method assumes that the two middle scores, "Diversified" and "Focused" respectively, are also indicators of partisanship, we also look at only the extreme scores "Dispersed" and "Biased" respectively to see if the results improve. 

In [29]:
only_extremes = discoursebias[(discoursebias.bias == "Biased") | (discoursebias.bias == "Dispersed")]
only_extremes.head(5)

,id,bias,hyperpartisanish,hyperpartisan_gt
20,0000250,Biased,true,true
56,0000527,Dispersed,false,true
97,0000870,Dispersed,false,true
100,0000888,Dispersed,false,true
125,0001071,Dispersed,false,true


In [30]:
res2 = classification_report(only_extremes["hyperpartisanish"], only_extremes["hyperpartisan_gt"])

In [31]:
print(res2)

              precision    recall  f1-score   support

       false       0.91      0.55      0.68      5035
        true       0.03      0.20      0.05       329

   micro avg       0.52      0.52      0.52      5364
   macro avg       0.47      0.37      0.37      5364
weighted avg       0.86      0.52      0.64      5364

